# Task 1 models

In [1]:
import os
from typing import Dict,Text
import pandas as pd
import numpy as np
import datetime

In [2]:
import tensorflow as tf
import tensorflow_recommenders as tfrs

In [3]:
%load_ext tensorboard

In [4]:
paths = {
    'Data' : os.path.join(os.getcwd(),'Data'),
}

In [5]:
df = pd.read_csv(os.path.join(paths['Data'],'amazon_co_ecommerce_sample.csv'))


users = pd.DataFrame({'user':df.manufacturer.unique()})
users['user_id'] = [i+1 for i in range(users.shape[0])]


pro = pd.DataFrame({'product':df.product_name.unique()})
pro['product_id'] = [i+1 for i in range(pro.shape[0])]

new_df = df.set_index('manufacturer').join(users.set_index('user')).set_index('product_name').join(pro.set_index('product'))

In [6]:
products = list(new_df.index)

products = tf.convert_to_tensor(products)

products = tf.data.Dataset.from_tensor_slices({"product_title" : products})

In [7]:
user_id = list(new_df.user_id)
for x in range(len(user_id)):
    user_id[x] = str(user_id[x])
user_id = np.array(user_id)
sells = tf.data.Dataset.from_tensor_slices(
    ({
       "product_title" : tf.cast(new_df.index.values,tf.string),
       "user_id" : tf.cast(user_id,tf.string),
    })
)

In [8]:
sells = sells.map(lambda x: {
    "product_title": x["product_title"],
    "user_id": x["user_id"],
})
products = products.map(lambda x: x["product_title"])

In [9]:
tf.random.set_seed(42)
shuffled = sells.shuffle(100_00, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_00)
test = shuffled.skip(80_00).take(20_00)

In [10]:
product_titles = products.batch(1_0)
user_ids = sells.batch(1_0).map(lambda x: x["user_id"])


unique_product_titles = np.unique(np.concatenate(list(product_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [11]:
user_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
    tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32)
])

In [12]:
product_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_product_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_product_titles) + 1, 32)
])

In [13]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=products.batch(32).map(product_model)
)

In [14]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [15]:
class RecommendationSystemModel(tfrs.Model):

  def __init__(self, user_model, product_model):
    super().__init__()
    self.product_model: tf.keras.Model = product_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    user_embeddings = self.user_model(features["user_id"])
    positive_product_embeddings = self.product_model(features["product_title"])
    return self.task(user_embeddings, positive_product_embeddings)

In [16]:
cached_train = train.shuffle(100_000).batch(32).cache()
cached_test = test.batch(32).cache()

In [17]:
model = RecommendationSystemModel(user_model, product_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [18]:
!rm -rf ./logs/

In [19]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [20]:
model.fit(cached_train, 
          epochs=3,
          validation_data=(cached_test),
          callbacks=[tensorboard_callback])

Epoch 1/3
250/250 [==============================] - 85s 331ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0011 - factorized_top_k/top_10_categorical_accuracy: 0.0020 - factorized_top_k/top_50_categorical_accuracy: 0.0039 - factorized_top_k/top_100_categorical_accuracy: 0.0096 - loss: 110.9028 - regularization_loss: 0.0000e+00 - total_loss: 110.9028 - val_factorized_top_k/top_1_categorical_accuracy: 5.0000e-04 - val_factorized_top_k/top_5_categorical_accuracy: 0.0015 - val_factorized_top_k/top_10_categorical_accuracy: 0.0015 - val_factorized_top_k/top_50_categorical_accuracy: 0.0035 - val_factorized_top_k/top_100_categorical_accuracy: 0.0100 - val_loss: 44.3122 - val_regularization_loss: 0.0000e+00 - val_total_loss: 44.3122
Epoch 2/3
250/250 [==============================] - 82s 329ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0367 - factorized_top_k

In [21]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 13272), started 8:10:29 ago. (Use '!kill 13272' to kill it.)

In [22]:
#model.evaluate(cached_test, return_dict=True)

In [23]:
def Predicter1(ind):
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

    index.index_from_dataset(
      tf.data.Dataset.zip((products.batch(100), products.batch(100).map(model.product_model)))
    )

    _, titles = index(tf.constant([ind]))
    return titles
titles = Predicter1("42")
print(f"Recommendations for user 42: {titles[0, :3]}")


Recommendations for user 42: [b'Thomas & Friends Trackmaster Gordon Engine'
 b"Thomas & Friends Trackmaster Gordon's Hill Track pack"
 b'Thomas & Friends Take-n-Play Straight & Curved Track Pack']


# Updated Models (Task 4)

In [24]:
user_model_2 = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
    tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
    tf.keras.layers.Dense(32,activation='relu')
])

In [25]:
product_model_2 = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
      vocabulary=unique_product_titles, mask_token=None),
    tf.keras.layers.Embedding(len(unique_product_titles) + 1, 32),
    tf.keras.layers.Dense(32,activation='relu')
])

In [26]:
model_2 = RecommendationSystemModel(user_model_2, product_model_2)
model_2.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [27]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [28]:
model_2.fit(cached_train, 
          epochs=10,
          validation_data=(cached_test),
          callbacks=[tensorboard_callback])

Epoch 1/10
250/250 [==============================] - 86s 340ms/step - factorized_top_k/top_1_categorical_accuracy: 1.2500e-04 - factorized_top_k/top_5_categorical_accuracy: 5.0000e-04 - factorized_top_k/top_10_categorical_accuracy: 0.0014 - factorized_top_k/top_50_categorical_accuracy: 0.0155 - factorized_top_k/top_100_categorical_accuracy: 0.0986 - loss: 110.9041 - regularization_loss: 0.0000e+00 - total_loss: 110.9041 - val_factorized_top_k/top_1_categorical_accuracy: 0.0010 - val_factorized_top_k/top_5_categorical_accuracy: 0.0065 - val_factorized_top_k/top_10_categorical_accuracy: 0.0145 - val_factorized_top_k/top_50_categorical_accuracy: 0.1120 - val_factorized_top_k/top_100_categorical_accuracy: 0.4280 - val_loss: 44.3421 - val_regularization_loss: 0.0000e+00 - val_total_loss: 44.3421
Epoch 2/10
250/250 [==============================] - 82s 329ms/step - factorized_top_k/top_1_categorical_accuracy: 0.7344 - factorized_top_k/top_5_categorical_accuracy: 0.7856 - factorized_top_k/t

In [29]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 13272), started 8:25:32 ago. (Use '!kill 13272' to kill it.)

In [30]:
def Predector2(ind):
    index = tfrs.layers.factorized_top_k.BruteForce(model_2.user_model)

    index.index_from_dataset(
      tf.data.Dataset.zip((products.batch(100), products.batch(100).map(model_2.product_model)))
    )

    _, titles = index(tf.constant([ind]))
    return titles

titles_2 = Predector2("42")
print(f"Recommendations for user 42: {titles_2[0, :3]}")

Recommendations for user 42: [b'Thomas & Friends Take-N-Play Harvey Engine'
 b'Thomas & Friends Take-n-Play Ferdinand Engine'
 b'Thomas & Friends Take-n-Play Stepney Engine']


In [32]:
new_df[new_df['user_id'] == 42]

,uniq_id,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers,user_id,product_id
Fisher Price Thomas & Friends Trackmaster Motorised Engine - Den,2a165629294785a493304cb43ee48c3a,£14.00,1 used,1.0,4.0,5.0 out of 5 stars,Die-Cast & Toy Vehicles > Toy Trains & Accesso...,http://www.amazon.co.uk/Thomas-Friends-Trackma...,This Trackmaster Den Motorised Engine is neve...,Technical Details Item Weight132 g Product Dim...,This Trackmaster Den Motorised Engine is neve...,http://www.amazon.co.uk/Thomas-Friends-Trackma...,NaN,very happy with Den // 5.0 // 26 Nov. 2015 // ...,NaN,42,7689
Fisher Price Wooden Thomas & Friends: Battery Operated Devious Diesel Y4109,450a47a28fa5b1ffb02b6d91e03946a4,£16.63,27 new,7.0,1.0,4.3 out of 5 stars,Die-Cast & Toy Vehicles > Toy Trains & Accesso...,http://www.amazon.co.uk/Thomas-Friends-Battery...,Little engineers will love watching Diesel tra...,Technical Details Item Weight181 g Product Dim...,Little engineers will love watching Diesel tra...,http://www.amazon.co.uk/Thomas-Friends-Railway...,NaN,nice brio loco // 5.0 // 23 Mar. 2014 // By\n ...,"{""seller""=>[{""Seller_name_1""=>""KM-Japan"", ""Sel...",42,7781
Fisher-Price Thomas & Friends Take-n-Play Cranky Docks,7e104001106c14ea9b8788f5f3cbaa8c,£21.75,28 new,122.0,3.0,4.6 out of 5 stars,Die-Cast & Toy Vehicles > Toy Trains & Accesso...,http://www.amazon.co.uk/Thomas-Friends-Take-n-...,Product Description Take-n-Play Cranky At the ...,Technical Details Item Weight1 Kg Product Dime...,Product Description Take-n-Play Cranky At the ...,http://www.amazon.co.uk/Thomas-Friends-Take-n-...,The picture shows the old model with 'square' ...,Mobile cargo transporter. // 5.0 // 12 Aug. 20...,"{""seller""=>[{""Seller_name_1""=>""Amazon.co.uk"", ...",42,7769
THOMAS & FRIENDS OLIVER 3+,0255e0900977ef81eb70611c03408961,NaN,2 new,1.0,3.0,5.0 out of 5 stars,Die-Cast & Toy Vehicles > Toy Trains & Accesso...,http://www.amazon.co.uk/Thomas-Friends-Wooden-...,THOMAS THE TANK ENGINE & FRIENDS,Technical Details Item Weight9 g Product Dimen...,THOMAS THE TANK ENGINE & FRIENDS,http://www.amazon.co.uk/Thomas-Friends-Wooden-...,NaN,Oliver Book..Thomas The Tank Engine // 5.0 // ...,"{""seller""=>[{""Seller_name_1""=>""musicandmedia"",...",42,7825
Thomas & Friends Captain,f50d624e185ce226cc52855cecc74bd4,NaN,2 new,4.0,1.0,4.3 out of 5 stars,Die-Cast & Toy Vehicles > Toy Vehicles & Acces...,http://www.amazon.co.uk/Thomas-Friends-Wooden-...,Product Description Compatible with other Thom...,Technical Details Item Weight227 g Product Dim...,Product Description Compatible with other Thom...,http://www.amazon.co.uk/Thomas-Friends-Take-n-...,NaN,Little Thomas fans will love it // 4.0 // 14 M...,"{""seller""=>[{""Seller_name_1""=>""PATRICK'S TOYS ...",42,7696
Thomas & Friends Minis 4cm Engine Wave 2 - Dino Gator,4a0a46d1abbfbdaa8b2e07dac87aca95,£2.65,NaN,1.0,1.0,5.0 out of 5 stars,Die-Cast & Toy Vehicles > Toy Trains & Accesso...,http://www.amazon.co.uk/Thomas-Friends-Minis-E...,Thomas and Friends Minis they are a collection...,Technical Details Item Weight9 g Product Dimen...,Thomas and Friends Minis they are a collection...,http://www.amazon.co.uk/Thomas-Friends-Minis-E...,NaN,Five Stars // 5.0 // 27 Nov. 2015 // By\n \...,"{""seller""=>{""Seller_name_1""=>""Tootally Thomas""...",42,7758
Thomas & Friends Minis 4cm Engine Wave 2 - Robot Charlie,95370580e723c5798b70aff7b4c5a664,£2.99,NaN,2.0,1.0,4.0 out of 5 stars,Die-Cast & Toy Vehicles > Toy Trains & Accesso...,http://www.amazon.co.uk/Thomas-Friends-Minis-E...,Thomas and Friends Minis they are a collection...,Technical Details Item Weight9 g Product Dimen...,Thomas and Friends Minis they are a collection...,http://www.amazon.co.uk/Toys-Thomas-Friends-Mi...,NaN,Thr

In [ ]:
results = {'user_id': [], 'First Model': [], 'Second Model' :[]}
for i in list(user_id):
    results['user_id'] .append(i)
    results['First Model'].append(Predicter1(str(i))[0, :3])
    results['Second Model'].append(Predector2(str(i))[0, :3])
pd.DataFrame(results)